<h1>Multiview Registration RANSAC</h1>

In [1]:
import open3d as o3d
import numpy as np 
import copy
from matplotlib import pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# Reference https://www.open3d.org/html/tutorial/Advanced/global_registration.html
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0, 0])
    #target_temp.paint_uniform_color([0, 0, 1])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
def get_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0, 0])
    #target_temp.paint_uniform_color([0, 0, 1])
    source_temp.transform(transformation)
    return source_temp + target_temp

In [4]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
# Load point clouds
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    source = o3d.io.read_point_cloud('./view_4_5_6_rgb_pcd_light_obj_dectection_colour_cropped.ply')
    target = o3d.io.read_point_cloud('./view_1_2_3_rgb_pcd_light_obj_dectection_colour_cropped.ply')
    source.estimate_normals()
    target.estimate_normals()
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [6]:
voxel_size = 15 # 20 = normal, 15 = final
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 15.000.
:: Estimate normal with search radius 30.000.
:: Compute FPFH feature with search radius 75.000.
:: Downsample with a voxel size 15.000.
:: Estimate normal with search radius 30.000.
:: Compute FPFH feature with search radius 75.000.


In [7]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [8]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 15.000,
   we use a liberal distance threshold 22.500.
RegistrationResult with fitness=7.868375e-01, inlier_rmse=1.037419e+01, and correspondence_set size of 2702
Access transformation to get result.


In [9]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result

In [10]:
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 6.000.
RegistrationResult with fitness=5.538407e-01, inlier_rmse=2.957880e+00, and correspondence_set size of 41841
Access transformation to get result.


In [11]:
output_pcd = get_registration_result(source, target, result_icp.transformation)
o3d.visualization.draw_geometries([output_pcd])
#o3d.io.write_point_cloud('./complete_point_cloud_ransac_colour.ply', output_pcd)

Downsample / outlier removal

pcd_ds = output_pcd.voxel_down_sample(voxel_size=20)
#o3d.visualization.draw_geometries([pcd_ds])
pcd_ds, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=50,std_ratio=2.0)
pcd_ds, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=50,std_ratio=2.0)
pcd_ds, ind = pcd_ds.remove_radius_outlier(nb_points=10, radius=30)
o3d.visualization.draw_geometries([pcd_ds])
#o3d.io.write_point_cloud('./complete_point_cloud_ransac_colour_downsampled.ply', pcd_ds)

temp = o3d.io.read_point_cloud('./complete_point_cloud_ransac_downsampled.ply')
o3d.visualization.draw_geometries([temp])